In [276]:
from scipy.optimize import linprog
import numpy as np
import pickle

### First define some utility pairings based on a scenario

In [108]:
def generate_utility_pairings(I,T):
    '''Create a dummy scenario statisfying valid weight properties'''
    impatient = np.random.rand(T,I+1)
    patient = np.random.rand(I,I+1)
    
    patient[:,0] = 0
    #make a few paris non-compatible
    pairing_weights = np.concatenate((impatient, patient), axis=0)
    for i in range(I+T):
        i,j = np.random.randint(0,I+T), np.random.randint(1,I+1)
        pairing_weights[i,j] = 0
    
    #make patient-patient pairings symmetric
    pairing_weights[T:,1:] = np.maximum(pairing_weights[T:,1:], pairing_weights[T:,1:].T)
    
    return pairing_weights

In [283]:
T = 5 #impatient agents
I = 5 #patient agents

pairing_weights = generate_utility_pairings(I,T)

### Now define several functions to convert our problem formulation to standard matrix form

In [189]:
def make_primal_constraint_matrix(I, T, c_len, pairing_weights):
    constraints = np.zeros((T+I, c_len))
    
    for a in range(0, T):
        constraint = np.zeros(pairing_weights.shape)
        constraint[a, :] = 1
        constraints[a,:] = constraint.reshape(constraint.shape[0] * constraint.shape[1])
    
    for B in range(0, I):
        constraint = np.zeros(pairing_weights.shape)
        constraint[B+T, 1:] += 1
        constraint[:, B+1] += 1
        constraints[B+T,:] = constraint.reshape(constraint.shape[0] * constraint.shape[1])
    
    return constraints

def make_dual_constraint_matrix(I, T, pairing_weights):
    constraints = np.zeros(((I+1)*T+I*I, T+I))
    inequalities = np.zeros(((I+1)*T+I*I,1))
    cix = 0
    
    #impatient constraints
    for t in range(T):    
        bixs = [i for i in range(T,T+I)]
        bixs.insert(0,t)
        
        for ix, i in enumerate(bixs): 
            constraints[cix, t] = -1
            constraints[cix,i] = -1
            inequalities[cix] = -1 * pairing_weights[t, ix]
            cix += 1
            
    #patient constraints
    for i in range(I):    
        for j in range(I):
            constraints[cix, T+i] -= 1
            constraints[cix, T+j] -= 1
            inequalities[cix] = -1 * pairing_weights[T+i, j+1]
            cix += 1

    return constraints, inequalities

### Find optimal pairings & utility based on the primal. Find optimal coefficients & utility based on the dual

In [119]:
def calculate_primal_solns(pairing_weights, I, T):

    c = -1 * pairing_weights.reshape(pairing_weights.shape[0] * pairing_weights.shape[1])
    constraints = make_primal_constraint_matrix(I, T, len(c), pairing_weights)
    b = np.ones((constraints.shape[0],1))

    solved_program = linprog(c, constraints, b)
    pairings = solved_program.x.reshape(pairing_weights.shape)
    utility_p = -1 * solved_program.fun
    
    return utility_p, pairings

def calculate_dual_solns(pairing_weights, I, T):

    constraints_d, inequalities_d = make_dual_constraint_matrix(I, T, pairing_weights)
    c_d = np.ones((T+I))

    solved_program = linprog(c_d, constraints_d, inequalities_d)
    coefficients = solved_program.x
    utility_d = solved_program.fun
    
    return utility_d, coefficients

### Assert utility found in primal & dual are equivalent (to precision error)

In [282]:
utility_p, _ = calculate_primal_solns(pairing_weights, I, T)
utility_d, _ = calculate_dual_solns(pairing_weights, I, T)

print(utility_d)
print(utility_p)

utility_d == utility_p

# assert round(utility_d, 10) == round(utility_p, 10), "Dual and Primal solutions are not equivilant"

4.830500111372682
4.830500111372682


True

# Testing
- Iterate over different patient/impatient scenarios and validate primal & dual produce same utility

In [278]:
for R in range(1,10):
    for S in range(1,10):
        pairing_weights = generate_utility_pairings(R,S)
        utility_1, _ = calculate_primal_solns(pairing_weights, R, S)
        utility_2, _ = calculate_dual_solns(pairing_weights, R, S)
        assert round(utility_1, 10) == round(utility_2, 10), "Dual and Primal solutions are not equivilant"

### Implement online assignment algorithm and check solution

In [273]:
def online_dual_assignment(betas, pairing_weights, I, T):
    betas = np.insert(betas, 0,0)
    II = [i for i in range(I+1)]
    II_membership = np.ones((I+1), np.bool)
    matches = []
    
    for t in range(0,T):
        shaddow_pairings = pairing_weights[t, :] - betas
        shaddow_pairings[~II_membership] = -1
        io = np.argmax(shaddow_pairings)
        
        if io == 0:
            matches.append((t,0))
        else:
            matches.append((t, io))
            II_membership[io] = 0
            
    for j in range(1,I+1):
        if II_membership[j]:
            II_membership_tmp = II_membership.copy()
            II_membership_tmp[j] = 0
            print('Weights:')
            print(pairing_weights[T+j-1, :])
            print('Betas')
            print(betas)

            shaddow_pairings = pairing_weights[T+j-1, :] - betas
            shaddow_pairings[~II_membership_tmp] = -1
            print('Weights - betas')
            print(shaddow_pairings)
            
            io = np.argmax(shaddow_pairings)

            if io != 0:
                matches.append((T+j-1, io))
                II_membership[io] = 0
            
            break
         
    pairings = np.zeros(pairing_weights.shape)
    utility = 0
    for match in matches:
        pairings[match[0], match[1]] = 1
        utility += pairing_weights[match[0], match[1]]
                
    return utility, pairings
    

In [226]:
T = 5 #impatient agents
I = 5 #patient agents

# pairing_weights = generate_utility_pairings(I,T)

In [277]:
pickleFile = open("debuginstance.txt", 'wb')
pickle.dump(pairing_weights, pickleFile)
pickleFile.close()

In [285]:
pairing_weights

array([[0.06799762, 0.76122642, 0.99577902, 0.24130629, 0.46976927,
        0.11021857],
       [0.10810557, 0.8030332 , 0.        , 0.5311666 , 0.96972489,
        0.3549381 ],
       [0.39339933, 0.3973132 , 0.46651218, 0.42004426, 0.89323478,
        0.47955169],
       [0.08387418, 0.        , 0.        , 0.        , 0.92890696,
        0.25040304],
       [0.03482951, 0.92624859, 0.52419646, 0.        , 0.0875872 ,
        0.20074078],
       [0.        , 0.56272317, 0.19681606, 0.24800205, 0.64391739,
        0.68286184],
       [0.        , 0.19681606, 0.30832998, 0.99882046, 0.71220159,
        0.6560531 ],
       [0.        , 0.24800205, 0.99882046, 0.77014179, 0.9719364 ,
        0.66701274],
       [0.        , 0.64391739, 0.71220159, 0.9719364 , 0.14205   ,
        0.8580919 ],
       [0.        , 0.68286184, 0.6560531 , 0.66701274, 0.8580919 ,
        0.        ]])

In [284]:
pairing_weights

array([[0.06799762, 0.76122642, 0.99577902, 0.24130629, 0.46976927,
        0.11021857],
       [0.10810557, 0.8030332 , 0.        , 0.5311666 , 0.96972489,
        0.3549381 ],
       [0.39339933, 0.3973132 , 0.46651218, 0.42004426, 0.89323478,
        0.47955169],
       [0.08387418, 0.        , 0.        , 0.        , 0.92890696,
        0.25040304],
       [0.03482951, 0.92624859, 0.52419646, 0.        , 0.0875872 ,
        0.20074078],
       [0.        , 0.56272317, 0.19681606, 0.24800205, 0.64391739,
        0.68286184],
       [0.        , 0.19681606, 0.30832998, 0.99882046, 0.71220159,
        0.6560531 ],
       [0.        , 0.24800205, 0.99882046, 0.77014179, 0.9719364 ,
        0.66701274],
       [0.        , 0.64391739, 0.71220159, 0.9719364 , 0.14205   ,
        0.8580919 ],
       [0.        , 0.68286184, 0.6560531 , 0.66701274, 0.8580919 ,
        0.        ]])

In [274]:
#Get knowlege of optimal Bis
up, pairings_actual = calculate_primal_solns(pairing_weights, I, T)
ud, coeffs = calculate_dual_solns(pairing_weights, I, T)

utility, pairings_test = online_dual_assignment(coeffs[T:], pairing_weights, I, T)

print('Optimal allocations from primal')
print(pairings_actual)
print('Online allocations')
print(pairings_test)

print('\nPrimal utility')
print(up)
print('Online utility')
print(utility)

Weights:
[0.         0.15315055 0.65299646 0.78726059 0.         0.97351918]
Betas
[0.         0.54760964 0.57348761 0.21377299 0.63911769 0.40003157]
Weights - betas
[ 0.         -1.         -1.          0.57348761 -0.63911769  0.57348761]
Optimal allocations from primal
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Online allocations
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]

Primal utility
5.116956110655744
Online utility
4.0778068486109245


In [141]:
T

2

In [75]:
[0,7,4] - betas

array([-0.80000001,  2.30000001,  2.30000001])

In [21]:


np.argmax([0,7,4] - betas)

1

In [620]:
pairings

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 0.],
       [0., 0., 0.]])